# Wstęp


## Wybrany temat

todo


## Pobieranie danych

todo


In [29]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime

In [30]:
def scrap_tweets(max_tweets_per_day, date_from, date_to):
    tweets = []

    date_from_2 = date_from + datetime.timedelta(days=1)

    while date_from != date_to:
        query = f'#fifaworldcup lang:en since:{date_from} until:{date_from_2}'
        new_tweets = []
        for _, tweet in enumerate(sntwitter.TwitterHashtagScraper(query, maxEmptyPages=5).get_items()):
            new_tweets.append(tweet)

            if len(new_tweets) >= max_tweets_per_day:
                break

        if len(new_tweets) < max_tweets_per_day:
            query = f'#worldcup lang:en since:{date_from} until:{date_from_2}'
            for _, tweet in enumerate(sntwitter.TwitterHashtagScraper(query, maxEmptyPages=5).get_items()):
                new_tweets.append(tweet)

                if len(new_tweets) >= max_tweets_per_day:
                    break

        if len(new_tweets) < max_tweets_per_day:
            query = f'#wc lang:en since:{date_from} until:{date_from_2}'
            for _, tweet in enumerate(sntwitter.TwitterHashtagScraper(query, maxEmptyPages=5).get_items()):
                new_tweets.append(tweet)

                if len(new_tweets) >= max_tweets_per_day:
                    break

        tweets.extend(new_tweets)
        date_from += datetime.timedelta(days=1)
        date_from_2 += datetime.timedelta(days=1)

    df = pd.DataFrame(tweets)

    return df

In [31]:
tweets = pd.read_csv("data_wc.csv", sep="\t")

print(f'Number of tweets: {len(tweets)}')

Number of tweets: 52927


# Przetwarzanie zebranych danych

## Wyodrębnienie potrzebnych danych


In [32]:
tweets = tweets[["date", "renderedContent"]]

tweets["date"] = tweets.apply(
    lambda x: datetime.datetime.strptime(x["date"], "%Y-%m-%d %H:%M:%S+00:00").date(), axis=1)

## Tokenizacja tweetów


In [33]:
from nltk.tokenize import word_tokenize

tweets["tokenizedContent"] = tweets.apply(
    lambda x: word_tokenize(x["renderedContent"]), axis=1)

## Usunięcie stopwords


In [34]:
from nltk.corpus import stopwords

english_stopwords = stopwords.words('english')

tweets["tokensWithoutStopwords"] = tweets.apply(lambda x: [
    word for word in x["tokenizedContent"] if word not in english_stopwords], axis=1)

## Lematyzacja


In [35]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

tweets["lemmatizedContent"] = tweets.apply(lambda x: [lemmatizer.lemmatize(
    word) for word in x["tokensWithoutStopwords"]], axis=1)

## Wyświetlenie bazy tweetów po przygotowaniu


In [36]:
display(tweets)

,date,renderedContent,tokenizedContent,tokensWithoutStopwords,lemmatizedContent
0,2022-09-01,FIFA World Cup Trophy arrives in Tehran dlvr.i...,"[FIFA, World, Cup, Trophy, arrives, in, Tehran...","[FIFA, World, Cup, Trophy, arrives, Tehran, dl...","[FIFA, World, Cup, Trophy, arrives, Tehran, dl..."
1,2022-09-01,Iran puts FIFA World Cup trophy on display for...,"[Iran, puts, FIFA, World, Cup, trophy, on, dis...","[Iran, puts, FIFA, World, Cup, trophy, display...","[Iran, put, FIFA, World, Cup, trophy, display,..."
2,2022-09-01,WATCH: Japanese referee Yoshimi Yamashita is s...,"[WATCH, :, Japanese, referee, Yoshimi, Yamashi...","[WATCH, :, Japanese, referee, Yoshimi, Yamashi...","[WATCH, :, Japanese, referee, Yoshimi, Yamashi..."
3,2022-09-01,Japanese referee Yoshimi Yamashita is set to m...,"[Japanese, referee, Yoshimi, Yamashita, is, se...","[Japanese, referee, Yoshimi, Yamashita, set, m...","[Japanese, referee, Yoshimi, Yamashita, set, m..."
4,2022-09-01,Japanese referee Yoshimi Yamashita is set to m...,"[Japanese, referee, Yoshimi, Yamashita, is, se...","[Japanese, referee, Yoshimi, Yamashita, set, m...","[Japanese, referee, Yoshimi, Yamashita, set, m..."
...,...,...,...,...,...
52922,2023-01-30,@RheaRipley_WWE First Aussie to win the #Royal...,"[@, RheaRipley_WWE, First, Aussie, to, win, th...","[@, RheaRipley_WWE, First, Aussie, win, #, Roy...","[@, RheaRipley_WWE, First, Aussie, win, #, Roy..."
52923,2023-01-30,Check out Christian Pulisic Fifa World Cup 202...,"[Check, out, Christian, Pulisic, Fifa, World, ...","[Check, Christian, Pulisic, Fifa, World, Cup, ...","[Check, Christian, Pulisic, Fifa, World, Cup, ..."
52924,2023-01-30,Check out Gabriel Jesus Brazil 2021-22 Panini ...,"[Check, out, Gabriel, Jesus, Brazil, 2021-22, ...","[Check, Gabriel, Jesus, Brazil, 2021-22, Panin...","[Check, Gabriel, Jesus, Brazil, 2021-22, Panin..."
52925,2023-01-30,Check out Erling Haaland Panini Donruss 2022 F...,"[Check, out, Erling, Haaland, Panini, Donruss,...","[Check, Erling, Haaland, Panini, Donruss, 2022...","[Check, Erling, Haaland, Panini, Donruss, 2022..."


# Analiza zebranych opinii


## Analiza za pomocą narzędzia NLTK Vader


In [37]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [39]:
sentiment_analyzer = SentimentIntensityAnalyzer()

tweets["vader_compound"] = tweets.apply(
    lambda x: sentiment_analyzer.polarity_scores(" ".join(x["lemmatizedContent"]))["compound"], axis=1)

# Podsumowanie


# Bibliografia
